<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/Majewska/Regresja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.datasets import make_regression

In [31]:
df = pd.read_csv('Salary Data (2).csv')
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


In [32]:
df.isna().sum()
df.dropna(inplace=True)

In [33]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

In [34]:
df.describe()

,Age,Years of Experience,Salary
count,373.000000,373.000000,373.000000
mean,37.431635,10.030831,100577.345845
std,7.069073,6.557007,48240.013482
min,23.000000,0.000000,350.000000
25%,31.000000,4.000000,55000.000000
50%,36.000000,9.000000,95000.000000
75%,44.000000,15.000000,140000.000000
max,53.000000,25.000000,250000.000000


In [35]:
le = LabelEncoder()

In [36]:
cols = ['Gender', 'Education Level', 'Job Title']

In [37]:
df[cols] = df[cols].apply(LabelEncoder().fit_transform)

In [38]:
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,1,0,159,5.0,90000.0
1,28.0,0,1,17,3.0,65000.0
2,45.0,1,2,130,15.0,150000.0
3,36.0,0,0,101,7.0,60000.0
4,52.0,1,1,22,20.0,200000.0


In [39]:
columns_to_convert = ['Age', 'Years of Experience']
df[columns_to_convert] = df[columns_to_convert].astype(float)

In [40]:
y = df['Salary']
X = df.drop('Salary', axis=1)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [43]:
reg = LinearRegression().fit(X_train, y_train)
reg.score(X, y)
reg.coef_
reg.intercept_
y_pred = reg.predict(X_test)

In [46]:
x_train = y_train
median_absolute_error(y_test, y_pred)

8858.308816896773

In [47]:
x_train = y_train
mean_absolute_error(y_test, y_pred)

11647.136609116484

In [49]:
r2_score(y_test, y_pred)

0.8857407952450546

In [51]:
huber = HuberRegressor().fit(X_train, y_train)
huber.score(X_train, y_train)
y_huber = huber.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [52]:
median_absolute_error(y_test, y_huber)

7875.886036166572

In [53]:
mean_absolute_error(y_test, y_huber)

10971.897430657022

In [54]:
r2_score(y_test, y_huber)

0.8944401207903664